In [26]:
import os
import glob
import psycopg2 as pg
import pandas as pd
from queries import *

In [27]:
conn = pg.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=student")
cur = conn.cursor()

In [28]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

In [29]:
song_files = get_files('data/song_data/')

In [30]:
file_path = song_files[0]
print(file_path)

/Users/piyushbhandari/Desktop/data-modelling-postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json
